In [1]:
import numpy as np
import librosa

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torch.nn.functional as F

from torchvision.utils import make_grid  # |   Utility stuff for plotting
import matplotlib.pyplot as plt          # |  <- I use this one a lot for plotting, seaborn is a good alternative
from matplotlib.image import imread      # |  it reads images... (png -> usable input (like a numpy array for ex))
import os
import random
from tqdm import tqdm  # | This one is a cute one for making a loading bar, I like it and we'll use it here

In [7]:
#load the CREMA-D AudioWAV dataset
def load_dataset(directory):
    dataset = []
    for filename in os.listdir(directory):
        if(filename.endswith('.wav')):
            filepath  = os.path.join(directory, filename)
            emotion_label = filename.split('_')[2]
            audio_tensor, _ = librosa.load(filepath, sr=None)
            audio_tensor = torch.tensor(audio_tensor, dtype=torch.float32)
            dataset.append([audio_tensor, emotion_label])
            
    return dataset

In [8]:
crema_d_directory = "./AudioWAV"
crema_d_data = load_dataset(crema_d_directory)
print(crema_d_data[0])

[tensor([ 0.0002, -0.0013, -0.0025,  ...,  0.0000,  0.0000,  0.0000]), 'ANG']


7442
